In [1]:
!pip install tmdbsimple

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [3]:
# Load API Credentials
with open(r'/Users/mmenj/.secret/tmdb_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate 
tmdb.API_KEY =  login['API Key']

In [4]:
def get_movie_with_rating(movie_id):
    #get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .release dictionaries
    info = movie.info()
    
    releases = movie.releases()
    
    #loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1'] == 'US':
            ## save a certification key in info w/ the certification
            info['certification'] = c['certification']
    return info

In [5]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 121.718,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [6]:
## testing our function by looping through a list of ids
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)

    except Exception as e: 
        errors.append([movie_id, e])

    
pd.DataFrame(results)
print(f"- Number of errors: {len(errors)}")
errors

- Number of errors: 1


[['tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=0c87bcffc60787c03e1a1cae71810544')]]

In [7]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_akas_chunk_001.csv.gz',
 'title_akas_chunk_002.csv.gz',
 'title_akas_chunk_003.csv.gz',
 'title_akas_chunk_004.csv.gz',
 'title_akas_chunk_005.csv.gz',
 'title_akas_chunk_006.csv.gz',
 'title_akas_chunk_007.csv.gz',
 'title_akas_chunk_008.csv.gz',
 'title_akas_chunk_009.csv.gz',
 'title_akas_chunk_010.csv.gz',
 'title_akas_chunk_011.csv.gz',
 'title_akas_chunk_012.csv.gz',
 'title_akas_chunk_013.csv.gz',
 'title_akas_chunk_014.csv.gz',
 'title_akas_chunk_015.csv.gz',
 'title_akas_chunk_016.csv.gz',
 'title_akas_chunk_017.csv.gz',
 'title_akas_chunk_018.csv.gz',
 'title_akas_chunk_019.csv.gz',
 'title_akas_chunk_020.csv.gz',
 'title_akas_chunk_021.csv.gz',
 'title_akas_chunk_022.csv.gz',
 'title_akas_chunk_023.csv.gz',
 'title_akas_chunk_024.csv.gz',
 'title_akas_chunk_025.csv.gz',
 'title_akas_chunk_026.csv.gz',
 'title_akas_chunk_027.csv.gz',
 'title_akas_chunk_028.csv.

In [8]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
                
        # Sets file's current position at offset.
        file.seek(0)
        
        # convert back to json.
        json.dump(file_data, file)

In [9]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')

In [10]:
YEARS_TO_GET = [2000,2001]

In [11]:
errors = [ ]

In [12]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/212 [00:00<?, ?it/s]

Movies from 2001: 0it [00:00, ?it/s]

- Total errors: 212
